## Node Classification with DGL
https://docs.dgl.ai/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py

In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"

In [2]:
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

## Loading Cora Dataset

In [3]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

/Users/haru256/.dgl/cora_v2.zip:   0%|          | 0.00/132k [00:00<?, ?B/s]

Extracting file to /Users/haru256/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [4]:
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})

In [5]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)


Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [7]:
g.ndata["feat"].size()

torch.Size([2708, 1433])

## Defining a Graph Convolutional Network (GCN)

In [8]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

## Training the GCN

In [9]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.122 (best 0.122), test acc: 0.114 (best 0.114)
In epoch 5, loss: 1.894, val acc: 0.400 (best 0.514), test acc: 0.399 (best 0.497)
In epoch 10, loss: 1.814, val acc: 0.476 (best 0.514), test acc: 0.492 (best 0.497)
In epoch 15, loss: 1.708, val acc: 0.590 (best 0.590), test acc: 0.602 (best 0.602)
In epoch 20, loss: 1.579, val acc: 0.650 (best 0.650), test acc: 0.643 (best 0.643)
In epoch 25, loss: 1.429, val acc: 0.684 (best 0.684), test acc: 0.677 (best 0.677)
In epoch 30, loss: 1.263, val acc: 0.708 (best 0.708), test acc: 0.697 (best 0.697)
In epoch 35, loss: 1.089, val acc: 0.712 (best 0.712), test acc: 0.717 (best 0.717)
In epoch 40, loss: 0.918, val acc: 0.730 (best 0.734), test acc: 0.738 (best 0.736)
In epoch 45, loss: 0.758, val acc: 0.744 (best 0.744), test acc: 0.754 (best 0.754)
In epoch 50, loss: 0.617, val acc: 0.758 (best 0.758), test acc: 0.767 (best 0.767)
In epoch 55, loss: 0.499, val acc: 0.764 (best 0.764), test acc: 0.773 (best 0